In [1]:
import pandas as pd
import os

# Parse IPC from Stats

In [2]:
resultsdir = "./results"
configs = [d for d in os.listdir(resultsdir) if os.path.isdir(os.path.join(resultsdir, d))]
# workloads = []
# for c in configs:
#     workloads.append([d for d in os.listdir(os.path.join(resultsdir, c)) if os.path.isdir(os.path.join(resultsdir, c, d))])
stats_per_config_workload = []

# for every config + workload directory
for c in configs:
    workloads = os.listdir(os.path.join("./results", c))
    for w in workloads:
        # find all files in the directory
        files = [f for f in os.listdir(os.path.join(resultsdir, c, w)) if os.path.isfile(os.path.join(resultsdir, c, w, f))]
        # find the stats file
        stat_files = [f for f in files if f.endswith('.stats')]
        # if there is a stats file
        if stat_files:
            for stat_file in stat_files:
                # if the stats_file has less than three lines skip it
                if len(open(os.path.join(resultsdir, c, w, stat_file)).readlines()) < 3:
                    continue
                
                extension = ''
                # if stats_file file name itself does not start with DDR4, parse it a bit
                if not stat_file.startswith('DDR4'):
                    # get the config name from the stats_file name
                    extension = '_'.join(stat_file.split('_')[:-1])
                    # prepend underscore to extension
                    extension = '_' + extension

                # read the stats file, name columns: 'name', 'value', 'description'
                df = pd.read_csv(os.path.join(resultsdir, c, w, stat_file), header=None).T
                df.columns = df.iloc[0]
                df.drop(0,inplace=True)
                # add a new column called 'config' with the config name
                df['config'] = c + extension
                # add a new column called 'workload' with the workload name
                df['workload'] = w
                # print the stats file
                # print('Config: {}, Workload: {}, Stats: {}'.format(c, w, df))
                # append the stats to the list
                df.reset_index(inplace=True, drop=True)
                stats_per_config_workload.append(df)
        else:
            print('Config: {}, Workload: {}, Stats: No stats file found'.format(c, w))

# concatenate all stats into one dataframe
stats = pd.concat(stats_per_config_workload)
stats_copy = stats.copy()

# remove from mc_only_stats the workloads that contain less than 7 dashes
mc_only_stats = stats[stats['workload'].str.count('-') >= 3]
sc_only_stats = stats[~stats['workload'].str.contains('-')].copy()

# expand the workload column into four columns by splitting using '-'
mc_only_stats[['wl0', 'wl1', 'wl2', 'wl3']] = mc_only_stats['workload'].str.split('-', expand=True)
sc_only_stats['ramulator.ipc'] = sc_only_stats['ramulator.record_insts_core_0'] / sc_only_stats['ramulator.record_cycs_core_0']

# for each ramulator.record_insts_core_i column, divide it by ramulator.record_cycs_core_i column and add it as ipci column
for i in range(0, 4):
    mc_only_stats['ipc{}'.format(i)] = mc_only_stats['ramulator.record_insts_core_{}'.format(i)] / mc_only_stats['ramulator.record_cycs_core_{}'.format(i)]

# write to csv mc_only_stats but only the config, workload and ipci columns
# mc_only_stats[['config', 'wl0', 'wl1', 'wl2', 'wl3', 'ipc0', 'ipc1', 'ipc2', 'ipc3']].to_csv('mc_only_stats.csv', index=False) 
mc_only_stats = mc_only_stats[['config', 'wl0', 'wl1', 'wl2', 'wl3', 'ipc0', 'ipc1', 'ipc2', 'ipc3']]

/tmp/ipykernel_1640568/713074044.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mc_only_stats[['wl0', 'wl1', 'wl2', 'wl3']] = mc_only_stats['workload'].str.split('-', expand=True)
/tmp/ipykernel_1640568/713074044.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mc_only_stats[['wl0', 'wl1', 'wl2', 'wl3']] = mc_only_stats['workload'].str.split('-', expand=True)
/tmp/ipykernel_1640568/713074044.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usin

# Compute Weighted Speedups

In [3]:
stats_parsed = mc_only_stats.copy()
stats_parsed['workload'] = stats_parsed['wl0']

baseline = sc_only_stats[sc_only_stats['config'] == 'Baseline.yaml']
baseline = baseline[['workload', 'ramulator.ipc']]
baseline.columns = ['workload', 'ramulator.baseline_ipc']
stats_parsed = pd.merge(stats_parsed, baseline, on='workload')

# for all ipc columns, divide by the baseline ipc
for i in range(0, 4):
    stats_parsed['normalized_ipc{}'.format(i)] = stats_parsed['ipc{}'.format(i)] / stats_parsed['ramulator.baseline_ipc']

stats_parsed['weighted_speedup'] = stats_parsed['normalized_ipc0'] + stats_parsed['normalized_ipc1'] + stats_parsed['normalized_ipc2'] + stats_parsed['normalized_ipc3'] 
stats_parsed = stats_parsed.sort_values(by=['config'])

# Graphene-RP Additional Perf. Overhead

In [4]:
def get_tmro(x):
  config_str = x
  config_str = config_str[config_str.find("OpenUntil"):]
  config_str = config_str.replace("OpenUntil", "")
  config_str = config_str.replace(".yaml", "")
  tmro = int(config_str)
  return tmro

In [5]:
graphene_data = stats_parsed.copy().loc[stats_parsed["config"].str.startswith("Graphene"), ["config", "workload", "weighted_speedup"]]
graphene_baseline = graphene_data.loc[graphene_data["config"] == "Graphene.yaml"]
graphene_data = pd.merge(graphene_data, graphene_baseline, on=["workload"])
graphene_data["normalized_ws"] = graphene_data["weighted_speedup_x"] / graphene_data["weighted_speedup_y"]
graphene_data = graphene_data.loc[graphene_data["config_x"] != "Graphene.yaml"]

graphene_data["tmro"] = graphene_data["config_x"].apply(get_tmro)
graphene_data = graphene_data[["tmro", "normalized_ws"]]

graphene_avg = graphene_data.groupby(["tmro"]).mean().reset_index()
graphene_avg["normalized_ws"] = 1 - graphene_avg["normalized_ws"]
graphene_avg.rename(columns={'normalized_ws':'Avg. Overhead'}, inplace=True)
graphene_max = graphene_data
graphene_max["normalized_ws"] = 1 - graphene_max["normalized_ws"]
graphene_max = graphene_max.groupby(["tmro"]).max().reset_index()
graphene_max.rename(columns={'normalized_ws':'Max. Overhead'}, inplace=True)

display(graphene_avg)
display(graphene_max)

,tmro,Avg. Overhead
0,36,0.013418
1,66,-0.004320
2,96,-0.006327
3,186,-0.004892
4,336,-0.001444
5,636,0.005995


,tmro,Max. Overhead
0,36,0.102196
1,66,0.065564
2,96,0.063659
3,186,0.050330
4,336,0.049573
5,636,0.046356


# PARA-RP

In [6]:
para_data = stats_parsed.copy().loc[stats_parsed["config"].str.startswith("PARA"), ["config", "workload", "weighted_speedup"]]
para_baseline = para_data.loc[para_data["config"] == "PARA.yaml"]
para_data = pd.merge(para_data, para_baseline, on=["workload"])
para_data["normalized_ws"] = para_data["weighted_speedup_x"] / para_data["weighted_speedup_y"]
para_data = para_data.loc[para_data["config_x"] != "PARA.yaml"]
para_data["tmro"] = para_data["config_x"].apply(get_tmro)
para_data = para_data[["tmro", "normalized_ws"]]
para_avg = para_data.groupby(["tmro"]).mean().reset_index()
para_avg["normalized_ws"] = 1 - para_avg["normalized_ws"]
para_avg.rename(columns={'normalized_ws':'Avg. Overhead'}, inplace=True)
para_max = para_data
para_max["normalized_ws"] = 1 - para_max["normalized_ws"]
para_max = para_max.groupby(["tmro"]).max().reset_index()
para_max.rename(columns={'normalized_ws':'Max. Overhead'}, inplace=True)

display(para_avg)
display(para_max)

,tmro,Avg. Overhead
0,36,0.031840
1,66,0.036227
2,96,0.045066
3,186,0.060861
4,336,0.079428
5,636,0.129264


,tmro,Max. Overhead
0,36,0.237591
1,66,0.134275
2,96,0.130798
3,186,0.146537
4,336,0.194413
5,636,0.316070
